## テキスト分析

- 2018-12-04
  -   Kunihiko Saito
  -   Yu Ishikawa

In [1]:
## 基本ライブラリの読み込み
import os
import sys

In [2]:
## 正規表現ライブラリの読み込み
import re

#　前処理　不必要な記号を削除
def preprocessing(document):

        lines = document.splitlines()
        processed_line = []

        horizontal_count = 0

        for line in lines:

            line =re.sub(r'[!~]', '', line) #半角記号を除去
            line =re.sub(r'[︰＠]', '', line) #全角記号を除去
            line = re.sub('\ufeff', '', line) # \ufeffを除去

            processed_line.append(line)

        return ''.join(processed_line)


In [8]:
#　ファイルの読み込み

code='shift-jis'
code='utf-8'#標準コード
file='./toshiba.txt'

with open(file, 'r', encoding=code) as f:
    row_documents= f.read()

docs=preprocessing(row_documents)
docs[:400]

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x93 in position 0: invalid start byte

### 分かち書き分析
名詞をカウント時

- 名詞で
features[1] == '一般'
とすると「電話」など、動詞としても使える名詞がカウントされない

- ーのコードが２種類存在　読めないものがある



In [ ]:
len(row_documents.splitlines())

In [ ]:
#! Mecab 'Hello World K'

In [ ]:
from natto import MeCab
def morphological(document):

        word_list = []
        #MeCabの形態素解析結果のフォーマット
        with MeCab('-F%f[0],%f[1],%f[6]') as mcb:
            for i, token in enumerate (mcb.parse(document, as_nodes=True)):
                features = token.feature.split(',')
                #名詞（一般）動詞（自立）、形容詞（自立）以外は除外
                if features[0] == '名詞' and features[1] == '一般' and features[2] != '':
                #if features[0] == '名詞'  and features[2] != '':
                    word_list.append(features[2])
                if features[0] == '動詞' and features[2] != '':
                    #word_list.append(features[2])  #動詞は無視
                    pass
                if features[0] == '形容詞' and features[1] == '自立' and features[2] != '':
                    word_list.append(features[2])
                
                if i%10000==0:
                    print(i, end=" ")
        return word_list


### 分かち書き一覧

### nattoがうまく動作しない場合

%env
で以下を確認
- 'PATH'に　C:\\Program Files\\MeCab\\bin　がある
- 'MECARC'が存在する
- 'MECAB_PATH'が存在する

存在しない場合は以下のコマンドで作成

os.environ['MECABRC']='C:\\Program Files\\MeCab\\etc\\mecabrc'

os.environ['MECAB_PATH']='C:\\Program Files\\MeCab\\bin\\libmecab.dll'

In [ ]:
#dict=morphological(docs)
dict=morphological(docs)

In [ ]:
len(dict)

### 単語の数え上げ

 itertoolsでさらっとカウント

In [ ]:
import itertools
# カウント処理のためのライブラリ
from collections import Counter
word_freq = Counter(itertools.chain(dict))

In [ ]:
#頻度の多いものを出力
word_freq.most_common(10)

### ソート

- dec_sort 降順にソート
- up_sort 昇順ソート


.most_commonを使えばソートの必要はない？

In [ ]:
dec_sort=sorted(word_freq.items(), key=lambda x: -x[1])
dec_sort[0:10]

In [ ]:
up_sort=sorted(word_freq.items(), key=lambda x: x[1])
up_sort[:10]

In [ ]:
# 単語の頻度降順に並べ替え
dic = []
for word_uniq in word_freq.most_common():
    dic.append(word_uniq[0])

In [ ]:
# 並べ替えた単語を表示
dic[:10]

In [ ]:
# 単語にIDを付与し、辞書の完成
dic_inv = {}
for i, word_uniq in enumerate(dic, start=1):
    dic_inv.update({word_uniq: i})

In [ ]:
# 辞書の中身を表示
dic_inv

In [ ]:
# 辞書のサイズを表示
len(dic_inv)

### 入力ファイルのからテキストを1行づつ取り出す

In [ ]:
lines = row_documents.splitlines()
len(lines)

In [ ]:
lines[0]

### テキストをコード化する
- lines[0]:最初のテキスト

In [ ]:
list=[]
with MeCab('-F%f[6]') as mcb:
    for token in mcb.parse(lines[0], as_nodes=True):        
        features = token.feature.split(',')
        key=features
        if(key[0] in dic_inv):
            list.append(dic_inv[key[0]])

print(list)                

### 全テキストをコード化
ディープラーニングの入力を作成

時間がかかる

In [ ]:
lists=[]

with MeCab('-F%f[6]') as mcb:
    for line in lines:
        list=[]    
        for token in mcb.parse(line, as_nodes=True):        
            features = token.feature.split(',')
            key=features
            if(key[0] in dic_inv):
                list.append(dic_inv[key[0]])
    lists.append(list)
    #print(list)

len(lists)            

In [ ]:
import numpy as np
nplist = np.array(lists)

In [ ]:
max(nplist[:])

In [5]:
import MeCab

### 再度、分かち書きを行い確認

In [6]:
import MeCab
m = MeCab.Tagger ("-Ochasen")
print(m.parse (lines[0]))

NameError: name 'lines' is not defined

### 以下ディープラーニングLSTM　正解データが必要

- 故障と関係あるテキストに１を紐づける

In [ ]:
## ライブラリの読み込み ##

# TensorFlowライブラリ
import tensorflow as tf
# TFLearnライブラリ
import tflearn
# データの前処理を行うライブラリ
from tflearn.data_utils import to_categorical, pad_sequences




In [ ]:
# テキストの単語をIDへ変換し配列へ格納
# 単語データの配列のサイズを合わせる
trainX = pad_sequences(lists, maxlen=50, value=0.)

# 正解データを配列へ格納
# 正解データのサイズを合わせる
#trainY =????
#trainY = to_categorical(trainY, nb_classes=2)

In [ ]:
trainX [2]

In [ ]:
len(trainX)

In [ ]:
## ニューラルネットワークの作成 ##


input_dim=75826

## 初期化
tf.reset_default_graph()

## 入力層の作成
net = tflearn.input_data([None, 32])

## 中間層の作成
# 単語埋め込み層
net = tflearn.embedding(net, input_dim=75826, output_dim=128)

# LSTMブロック
net = tflearn.lstm(net, 128, dropout=0.5)

## 出力層の作成 
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001, loss='categorical_crossentropy')

In [ ]:
## 4．モデルの作成（学習） ##
# 学習の実行
model = tflearn.DNN(net)
model.fit(trainX, trainY, n_epoch=50, batch_size=32, validation_set=0.2, shuffle=True, show_metric=True)

In [ ]:
from tflearn.datasets import imdb
train, test, _ = imdb.load_data(path='imdb.pkl', n_words=10000,valid_portion=0.1)

In [ ]:
len(test)

In [ ]:
%env
